In [1]:
#載入模組
import zipfile
import os
import csv
from dateutil.relativedelta import relativedelta
import datetime
import time
import os
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.chrome.options import Options

#### 瀏覽器設定

打開瀏覽器進入銀行局網頁

In [2]:
executable_path = r'chromedriver.exe' 

option = Options()
option.add_argument("--disable-notifications")
option.add_argument("--disable-popup-blocking")
# option.add_argument('--headless=new') # 打開背景模式
driver = webdriver.Chrome(executable_path, options=option)
driver2 = webdriver.Chrome(executable_path, options=option)


# 打開銀行局跨類別查詢
target_url = 'https://survey.banking.gov.tw/statis/webMain.aspx?sys=100&funid=defqry2'
driver.get(target_url)
driver.switch_to.frame('qry12')
# 打開銀行局自選統計項查詢
target_url2 = 'https://survey.banking.gov.tw/statis/webMain.aspx?sys=100&funid=defqry'
driver2.get(target_url2)
driver2.switch_to.frame('qry11')
driver2.find_element(By.XPATH, '//*[@id="c31"]/td/a').click()
driver2.find_element(By.XPATH, '//*[@id="c32"]/td/a').click()
driver2.switch_to.default_content()
driver2.switch_to.frame('qry12')

C:\Users\Chance\AppData\Local\Temp\ipykernel_55764\4078568938.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path, options=option)
C:\Users\Chance\AppData\Local\Temp\ipykernel_55764\4078568938.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver2 = webdriver.Chrome(executable_path, options=option)


#### 設定時間年月範圍

In [3]:
#輸入西元再轉換成民國，會捉取前一年的資料
input_year = str(int(input("選擇年度(西元): ")) - 1911)
input_month = str(input("選擇月份: "))

previous_month = str(int(input_month) - 1)
previous_year = str(int(input_year) - 1)

last_month = str(12)      #?

網頁一（跨類別查詢）

In [4]:
# 選擇資料"統計期"的開始時閶
driver.find_element(
    By.XPATH, f'//*[@id="yyf"]//option[text()="{previous_year}"]').click()  #年份
driver.find_element(
    By.XPATH, f'//*[@id="mmf"]//option[text()="{input_month}"]').click()    #月份

# 選擇資料"統計期"的完結時間
driver.find_element(
    By.XPATH, f'//*[@id="yyt"]//option[text()="{input_year}"]').click()     #年份
driver.find_element(
    By.XPATH, f'//*[@id="mmt"]//option[text()="{input_month}"]').click()    #月份

# 年月別選擇"月"
driver.find_element(By.XPATH, '//*[@id="cycle"]/option[1]').click()

# 輸出格式選擇"網頁"
driver.find_element(By.XPATH, '//*[@id="outkind"]/option[10]').click()

網頁二（自選統計項查詢）

In [5]:
# 選擇資料"統計期"的開始時閶
driver2.find_element(
    By.XPATH, f'//*[@id="yyf"]//option[text()="{previous_year}"]').click()
driver2.find_element(
    By.XPATH, f'//*[@id="mmf"]//option[text()="{input_month}"]').click()

# 選擇資料"統計期"的完結時間
driver2.find_element(
    By.XPATH, f'//*[@id="yyt"]//option[text()="{input_year}"]').click()
driver2.find_element(
    By.XPATH, f'//*[@id="mmt"]//option[text()="{input_month}"]').click()

# 年月別選擇"月"
driver2.find_element(By.XPATH, '//*[@id="cycle"]/option[1]').click()

# 輸出格式選擇"網頁"
driver2.find_element(By.XPATH, '//*[@id="outkind"]/option[5]').click()

#### 點選各網頁需要的資料

In [6]:
# 跨類別查詢
options_xpath = [#1-5
                 '//*[@id="226217010:0/1/0/$code1$/0/"]', #月底餘額(不含郵匯轉存款)
                 '//*[@id="folder4"]/tbody/tr/td[1]/a', #打開月底餘額(不含郵匯轉存款)細項
                 '//*[@id="226217010:3/1/0/$code1$/0/"]', #活期性存款
                 '//*[@id="226217010:1/1/0/$code1$/0/"]', #外匯存款
                 #6-10
                 '/html/body/form/table[3]/tbody/tr/td[1]/table[26]/tbody/tr/td[2]/input', #月底餘額(不含催收款)
                 '/html/body/form/table[3]/tbody/tr/td[1]/table[35]/tbody/tr/td[1]/a/img', #打開月底餘額(不含郵匯轉存款)下細項
                 '/html/body/form/table[3]/tbody/tr/td[1]/table[42]/tbody/tr/td[2]/input', #消費者貸款餘額-金額
                 '/html/body/form/table[3]/tbody/tr/td[1]/table[42]/tbody/tr/td[1]/a/img', #打開金額
                 '/html/body/form/table[3]/tbody/tr/td[1]/table[43]/tbody/tr/td[2]/input', #購置住宅貸款
                 '/html/body/form/table[3]/tbody/tr/td[1]/table[47]/tbody/tr/td[2]/input', #其他個人消費貸款(不含信用卡循環信用)
                 '/html/body/form/table[3]/tbody/tr/td[1]/table[48]/tbody/tr/td[2]/input', #授信總額
                 #11-15
                 '/html/body/form/table[3]/tbody/tr/td[1]/table[31]/tbody/tr/td[1]/a/img',
                 '/html/body/form/table[3]/tbody/tr/td[1]/table[33]/tbody/tr/td[2]/input', #民營企業放款餘額
                 '/html/body/form/table[3]/tbody/tr/td[1]/table[34]/tbody/tr/td[2]/input', #中小企業放款餘額
                 '/html/body/form/table[3]/tbody/tr/td[1]/table[79]/tbody/tr/td[2]/input', #流通金融卡數
                 '/html/body/form/table[3]/tbody/tr/td[1]/table[80]/tbody/tr/td[2]/input', #ATM裝設臺數
                 '/html/body/form/table[3]/tbody/tr/td[1]/table[81]/tbody/tr/td[2]/input', #金融卡交易次數
                 #16-20
                 #"/html/body/form/table[3]/tbody/tr/td[1]/table[86]/tbody/tr/td[2]/a/font/label",   #稅前損益
                 "/html/body/form/table[3]/tbody/tr/td[1]/table[86]/tbody/tr/td[1]/a/img",   #稅前損益細項
                 #"/html/body/form/table[3]/tbody/tr/td[1]/table[87]/tbody/tr/td[2]/a/font/label",   #收益
                 "/html/body/form/table[3]/tbody/tr/td[1]/table[87]/tbody/tr/td[1]/a/img",   #收益細項
                 "/html/body/form/table[3]/tbody/tr/td[1]/table[88]/tbody/tr/td[2]/font/label",   #利息收入
                 "/html/body/form/table[3]/tbody/tr/td[1]/table[89]/tbody/tr/td[2]/font/label",   #手續費收入
                 "/html/body/form/table[3]/tbody/tr/td[1]/table[90]/tbody/tr/td[2]/font/label",   #FVPL利益
                 "/html/body/form/table[3]/tbody/tr/td[1]/table[92]/tbody/tr/td[1]/a/img", #點開費損（累計數）
                 "/html/body/form/table[3]/tbody/tr/td[1]/table[95]/tbody/tr/td[2]/input", #FVPL損失
                 "/html/body/form/table[3]/tbody/tr/td[1]/table[91]/tbody/tr/td[2]/font/label",   #其他收益
                 "/html/body/form/table[3]/tbody/tr/td[1]/table[87]/tbody/tr/td[2]/input", #總營業收入
                 #26-30
]

# 點擊options_xpath的按鈕
for i in options_xpath:
    driver.find_element(By.XPATH, i).click()

# 從網頁取得銀行列表: local_list = ['本國銀行', '＊金控子銀行', '＊非金控子銀行', '◎公股銀行', '◎非公股銀行', '004 臺灣銀行', ...]
local_list = driver.find_element(By.XPATH, '//*[@id="sublist"]').text
local_list = local_list.split("\n")
local_list.remove("810 寶華商業銀行") # 810與星展銀行相同，保留星展銀行

# 建立銀行號碼對應銀行名稱的dict: local_dict = {'004': '臺灣銀行', '005': '臺灣土地銀行', ...}
local_dict = {}
for i in local_list[5:]:
    local_dict[i[:3]] = i[4:]

In [7]:
# 自選統計查詢
options_xpath2 = [#21-25
                  '//*[@id="226217490:$code1$/5/0/$cod21$/0/"]',
                  '//*[@id="226217490:$code1$/2/0/$cod21$/0/"]',
                  '//*[@id="226217490:$code1$/1/0/$cod21$/0/"]',
                  '//*[@id="310311000:3"]',
                  '//*[@id="310311000:4"]',
                  '//*[@id="320011000:20000"]',
]

# 點擊options_xpath2的按鈕
for i in options_xpath2:
    driver2.find_element(By.XPATH, i).click()

In [9]:
print(local_list[7-1][:3])

005


#### 抓取資料

In [10]:
# 將各銀行資料存入df
df = {}     #df不是dataframe, 是dict
for i in range(6, len(local_list)+1):
    #選擇第一個銀行
    if i == 6:
        driver.find_element(By.XPATH, f'//*[@id="sublist"]/option[{i}]').click() #選擇銀行
        driver2.find_element(By.XPATH, f'//label[text()="{local_list[i-1]}"]').click() #選擇銀行

    #清除上一個銀行的選項並選擇下一間銀行
    else:
        chwd = driver.window_handles
        driver.switch_to.window(chwd[0])
        driver.switch_to.frame('qry12')
        driver.find_element(By.XPATH, '/html/body/form/table[3]/tbody/tr/td[5]/div/a').click() #全清前選項
        driver.find_element(By.XPATH, f'//*[@id="sublist"]/option[{i}]').click() #選擇銀行

        chwd2 = driver2.window_handles
        driver2.switch_to.window(chwd2[0])
        driver2.switch_to.frame('qry12')
        driver2.find_element(By.XPATH, f'//label[text()="{local_list[i-2]}"]').click() #全清前選項
        driver2.find_element(By.XPATH, f'//label[text()="{local_list[i-1]}"]').click() #選擇銀行

    driver.find_element(By.XPATH, '//*[@id="center"]').click() #查詢
    driver2.find_element(By.XPATH, '//*[@id="center"]').click() #查詢

    # switch tab
    chwd = driver.window_handles
    driver.switch_to.window(chwd[1])
    chwd2 = driver2.window_handles
    driver2.switch_to.window(chwd2[1])

    # 在df中新增key-value pair: '銀行代碼'= []
    df[local_list[i-1][:3]] = []
    index_len = driver.find_elements(By.XPATH, '/html/body/form/table[2]/tbody//tr')
    index_len2 = driver2.find_elements(By.XPATH, '/html/body/form/table[2]/tbody/tr')

    # 抓取各銀行的每個指標的資料並續行放進對應的list
    for j in range(2, len(index_len)+1):
        df[local_list[i-1][:3]].append(driver.find_element(By.XPATH, f'/html/body/form/table[2]/tbody/tr[{j}]').text)
    for j in range(3, len(index_len2)+1):
        df[local_list[i-1][:3]].append(driver2.find_element(By.XPATH, f'/html/body/form/table[2]/tbody/tr[{j}]').text)

driver.quit()

In [11]:
df[local_list[7][:3]]

['月底餘額(不含郵匯轉存款) 3,453,753 3,453,818 3,500,597 3,506,185 3,527,451 3,492,735 3,570,345 3,579,274 3,603,794 3,591,935 3,545,807 3,598,512 3,648,329',
 '月底餘額--活期性存款 1,754,239 1,761,187 1,758,099 1,772,717 1,745,026 1,725,139 1,772,846 1,772,574 1,739,925 1,755,361 1,766,891 1,789,121 1,779,620',
 '月底餘額--外匯存款 542,658 535,540 564,536 552,673 578,169 580,357 598,955 599,688 651,361 613,694 549,068 567,745 586,423',
 '月底餘額(不含催收款) 2,453,937 2,490,518 2,521,351 2,529,767 2,499,464 2,484,474 2,509,608 2,527,009 2,595,608 2,607,848 2,584,262 2,608,249 2,746,222',
 '對企業放款餘額--對民營事業 1,020,695 1,011,764 1,034,153 1,019,924 1,019,910 1,039,796 1,059,954 1,073,409 1,100,186 1,098,733 1,072,768 1,056,736 1,083,233',
 '對企業放款餘額--對中小企業 826,240 824,182 823,966 824,884 819,959 823,955 828,780 832,805 842,316 839,609 836,315 829,482 832,810',
 '消費者貸款餘額--金額 666,478 666,038 669,917 672,952 676,656 678,332 680,709 681,468 683,495 684,036 686,457 690,961 689,581',
 '消費者貸款餘額--金額-購置住宅貸款 586,019 586,358 591,077 594,

信用卡資料

In [12]:
# 信用卡資料
chwd2 = driver2.window_handles
driver2.switch_to.window(chwd2[0])
driver2.switch_to.frame('qry11')
driver2.find_element(By.XPATH, '//*[@id="c12"]/td/a').click()
driver2.switch_to.default_content()
driver2.switch_to.frame('qry12')

# Start
driver2.find_element(By.XPATH, f'//*[@id="yyf"]//option[text()="{previous_year}"]').click()
driver2.find_element(By.XPATH, f'//*[@id="mmf"]//option[text()="{input_month}"]').click()

# End
driver2.find_element(By.XPATH, f'//*[@id="yyt"]//option[text()="{input_year}"]').click()
driver2.find_element(By.XPATH, f'//*[@id="mmt"]//option[text()="{input_month}"]').click()

# 年月別選擇"月"
driver2.find_element(By.XPATH, '//*[@id="cycle"]/option[1]').click()

# 輸出格式選擇"網頁"
driver2.find_element(By.XPATH, '//*[@id="outkind"]/option[10]').click()

# 從網頁取得銀行列表: local_list2 = ['004 臺灣銀行', '005 臺灣土地銀行', ...]
local_list2 = driver2.find_element(By.XPATH, "/html/body/form/table[3]/tbody/tr[2]/td[2]").text
local_list2 = local_list2.split("外國銀行在臺分行")
local_list2 = local_list2[0].split("\n")[7:-1]
local_list2.remove("810 寶華商業銀行")


# 信用卡資料
options_xpath3 = [
                  '//*[@id="226217350:0/0/0/$code1$/0/"]', #取消每月發卡數(張)
                  '//*[@id="226217360:0/0/0/$code1$/0/"]', #取消每月停卡數(張)
                  '//*[@id="226217370:0/0/0/$code1$/0/"]', #取消循環信用餘額
                  '//*[@id="226217380:0/0/0/$code1$/0/"]', #取消本月簽賬金額
                  '//*[@id="226217390:0/0/0/$code1$/0/"]', #取消本月預借現金金額
                  '//*[@id="226217420:0/0/0/$code1$/0/"]', #取消備抵呆帳提足率(%)
                  '//*[@id="226217430:0/0/0/$code1$/0/"]', #取消本月轉銷呆帳金額
                  '//*[@id="226217440:0/0/0/$code1$/0/"]', #取消循環信用利息收入
                  '//*[@id="226217450:0/0/0/$code1$/0/"]', #取消簽帳手續費收入
                  '//*[@id="226217460:0/0/0/$code1$/0/"]', #取消預借現金手續費
                  '//*[@id="226217470:0/0/0/$code1$/0/"]', #取消收單特約商店家數
                  '//*[@id="320011000:99999"]', #取消總計
                  '//*[@id="320011000:20000"]', #取消本國銀行
                  '//*[@id="320011000:999"]', #取消外國銀行在台分行
                  '//*[@id="320011000:10001"]', #取消信用合作社
                  '//*[@id="320011000:30000"]', #取消信託投資公司
                  '//*[@id="320011000:50000"]', #取消信用卡公司
]

# 按下options_xpath3的按鈕
for i in options_xpath3:
    driver2.find_element(By.XPATH, i).click()

In [13]:
# 捉信用卡資料並加到df中對應的list的最後
for i in range(len(local_list2)):
    #第一間銀行
    if i == 0:
        driver2.find_element(By.XPATH, f'//label[text()="{local_list2[i]}"]').click() #選擇銀行

    #清除上一個銀行的選項並選擇下一間銀行    
    else:
        chwd2 = driver2.window_handles
        driver2.switch_to.window(chwd2[0])
        driver2.switch_to.frame('qry12')
        driver2.find_element(By.XPATH, f'//label[text()="{local_list2[i-1]}"]').click() #清除上一個選項
        driver2.find_element(By.XPATH, f'//label[text()="{local_list2[i]}"]').click() #選擇銀行
    driver2.find_element(By.XPATH, '//*[@id="center"]').click() #查詢
    chwd2 = driver2.window_handles
    driver2.switch_to.window(chwd2[1])

    index_len2 = driver2.find_elements(By.XPATH, '/html/body/form/table[2]/tbody/tr')
    # 抓資料並append到對應的list
    for j in range(2, len(index_len2)+1):
        df[local_list2[i][:3]].append(driver2.find_element(By.XPATH, f'/html/body/form/table[2]/tbody/tr[{j}]').text)

driver2.quit()

In [14]:
print(df[local_list2[7][:3]])

['月底餘額(不含郵匯轉存款) 2,590,317 2,571,026 2,582,442 2,588,302 2,576,171 2,630,894 2,651,973 2,678,297 2,692,615 2,748,105 2,738,331 2,709,274 2,718,643', '月底餘額--活期性存款 955,949 967,978 971,718 976,778 975,600 953,574 972,494 1,002,623 966,947 963,737 961,835 969,137 976,293', '月底餘額--外匯存款 949,821 928,820 947,958 969,938 960,227 1,003,805 1,009,698 995,290 1,042,186 1,088,062 1,036,930 992,462 989,019', '月底餘額(不含催收款) 1,668,380 1,640,459 1,656,398 1,671,531 1,694,138 1,718,141 1,705,784 1,703,267 1,717,792 1,726,264 1,719,464 1,705,682 1,701,812', '對企業放款餘額--對民營事業 454,068 446,416 444,176 445,892 456,136 462,152 449,539 435,589 440,559 441,369 433,582 420,899 415,390', '對企業放款餘額--對中小企業 262,825 263,477 265,479 271,075 273,032 274,545 272,938 271,648 274,912 277,968 276,153 271,575 275,307', '消費者貸款餘額--金額 579,781 584,897 589,426 593,763 598,520 600,231 602,983 603,286 604,681 607,764 613,214 616,283 616,590', '消費者貸款餘額--金額-購置住宅貸款 523,282 528,841 533,867 537,937 541,990 544,179 547,431 548,363 550,293 554

#### 資料整理

設定欄位名稱

In [16]:
import datetime
from dateutil.relativedelta import relativedelta
# 轉換input_year的資料，從民國到西元
input_year = str(int(input_year) + 1911)
previous_year = str(int(previous_year) + 1911)
input_date = input_year+"/"+input_month+"/"+"01"
datetime = datetime.datetime.strptime(input_date, '%Y/%m/%d')

# 建立list"columns"並用作後續dataframe的column
# ['銀行別', '2022年1月', '2022年2月', ..., '2023年1月']
columns = ["銀行別"]
for i in range(12, -1, -1):
    past_date = datetime - relativedelta(months=i)
    columns.append(str(past_date.year)+"年"+str(past_date.month)+"月")

In [17]:
print(columns)

['銀行別', '2022年1月', '2022年2月', '2022年3月', '2022年4月', '2022年5月', '2022年6月', '2022年7月', '2022年8月', '2022年9月', '2022年10月', '2022年11月', '2022年12月', '2023年1月']


In [18]:
# 把str去掉","再變成float
def number_format(x):
    try:
        return float(x.replace(",", ""))
    except ValueError:
        return None

將指標成DataFrame格式

In [19]:
# 每個指標建立一個dict
總存款月底餘額 = {} #1
活期性存款月底餘額 = {} #2
外匯存款餘額 = {} #5
總放款餘額 = {} #7
消費者貸款餘額 = {} #8
房貸餘額 = {} #9
其他個人消費貸款餘 = {} #10
授信總額 = {} #extra
民營企業放款餘額 = {} #11
中小企業放款餘額 = {} #12
ATM裝設臺數 = {} #13
金融卡交易次數 = {} #14
流通金融卡數 = {} #15
總利息收入 = {} #16
總手續費收入 = {} #17
# 淨公平市價變動利益 = {} #18
# 淨公平市價變動費損 = {} #18
淨公平市價變動損益 = {} #18
其他收益 = {} #19
總營業收入 = {} #20
累計稅前損益 = {} #21
OBU累計稅前損益 = {} #22
國外分行累計稅前損益 = {} #23
信用卡有效卡數 = {} #24
信用卡流通卡數 = {} #25
活卡率 = {} #26
信用卡逾期帳款比率 = {} #34

# 從df續個打開每個銀行的數據，按行去掉數據名字，並放到對應的的指標dict
# df = {'004':[], '005':[], ...}
for i in df:
    總存款月底餘額[i] = df[i][0].split()[1:]
    活期性存款月底餘額[i] = df[i][1].split()[1:]
    外匯存款餘額[i] = df[i][2].split()[1:]
    總放款餘額[i] = df[i][3].split()[1:]
    民營企業放款餘額[i] = df[i][4].split()[1:]
    中小企業放款餘額[i] = df[i][5].split()[1:]
    消費者貸款餘額[i] = df[i][6].split()[1:]
    房貸餘額[i] = df[i][7].split()[1:]
    其他個人消費貸款餘[i] = df[i][8].split()[1:]
    授信總額[i] = df[i][9].split()[1:]
    流通金融卡數[i] = df[i][10].split()[1:]
    ATM裝設臺數[i] = df[i][11].split()[1:]
    金融卡交易次數[i] = df[i][12].split()[1:]
    總營業收入[i] = df[i][13].split()[1:]
    總利息收入[i] = df[i][14].split()[1:]
    總手續費收入[i] = df[i][15].split()[1:]
    try:
        淨公平市價變動損益[i] = [str(int(x.replace(",", "")) - int(y.replace(",", ""))) \
                        for x, y in zip(df[i][16].split()[2:],df[i][18].split()[2:])]
    except:
        淨公平市價變動損益[i] = df[i][18].split()[2:]
    其他收益[i] = df[i][17].split()[1:]
    累計稅前損益[i] = df[i][19].split()[3:]
    OBU累計稅前損益[i] = df[i][20].split()[3:]
    國外分行累計稅前損益[i] = df[i][21].split()[3:]
    try:
        信用卡流通卡數[i] = df[i][22].split()[1:]
        信用卡有效卡數[i] = df[i][23].split()[1:]
        try:
            活卡率[i] = [(int(x.replace(",", ""))/int(y.replace(",", ""))) \
                            for x, y in zip(df[i][23].split()[1:],df[i][22].split()[1:])]
        except:
            活卡率[i] = df[i][22].split()[1:]
        信用卡逾期帳款比率[i] = df[i][24].split()[1:]
    except IndexError:
        pass

ratio_list = [總存款月底餘額, 活期性存款月底餘額, 外匯存款餘額, 總放款餘額, 民營企業放款餘額, 中小企業放款餘額, 消費者貸款餘額,
              房貸餘額, 其他個人消費貸款餘, 授信總額, 流通金融卡數, ATM裝設臺數, 金融卡交易次數, 總營業收入, 總利息收入, 
              總手續費收入, 淨公平市價變動損益, 其他收益, 累計稅前損益, OBU累計稅前損益, 國外分行累計稅前損益, 信用卡流通卡數,
              信用卡有效卡數, 活卡率, 信用卡逾期帳款比率]
ratio_name = ["總存款月底餘額", "活期性存款月底餘額", "外匯存款餘額", "總放款餘額", "民營企業放款餘額", "中小企業放款餘額", "消費者貸款餘額",
              "房貸餘額", "其他個人消費貸款餘", "授信總額", "流通金融卡數", "ATM裝設臺數", "金融卡交易次數", "總營業收入", "總利息收入", 
              "總手續費收入", "淨公平市價變動損益", "其他收益", "累計稅前損益", "OBU累計稅前損益", "國外分行累計稅前損益", "信用卡流通卡數",
              "信用卡有效卡數", "活卡率", "信用卡逾期帳款比率"]

In [20]:
print(總存款月底餘額)

{'004': ['4,215,393', '4,254,275', '4,316,862', '4,346,140', '4,415,987', '4,434,159', '4,480,116', '4,487,632', '4,547,375', '4,621,764', '4,599,292', '4,670,608', '4,623,220'], '005': ['2,706,834', '2,687,970', '2,711,632', '2,708,190', '2,666,599', '2,706,520', '2,614,252', '2,612,200', '2,617,325', '2,582,141', '2,563,067', '2,594,105', '2,543,070'], '006': ['3,453,753', '3,453,818', '3,500,597', '3,506,185', '3,527,451', '3,492,735', '3,570,345', '3,579,274', '3,603,794', '3,591,935', '3,545,807', '3,598,512', '3,648,329'], '007': ['2,964,288', '3,008,880', '3,054,219', '3,105,435', '3,104,472', '3,098,871', '3,081,837', '3,098,359', '3,111,401', '3,185,255', '3,200,242', '3,289,898', '3,278,573'], '008': ['2,828,722', '2,844,616', '2,859,143', '2,890,644', '2,942,953', '2,900,734', '2,927,407', '2,879,237', '2,852,389', '2,844,098', '2,901,161', '2,952,377', '2,917,006'], '009': ['2,187,738', '2,210,755', '2,184,282', '2,209,120', '2,215,740', '2,241,815', '2,215,330', '2,244,263

In [21]:
# 建立一個空dict來放dataframe: ratio_dict={"總存款月底餘額":df, "活期性存款月底餘額":df, ...}
ratio_dict = {}
for i in range(len(ratio_list)):
    ratio_df = ratio_list[i]
    ratio_df = pd.DataFrame(ratio_df.items(), columns=[columns[0], "others"])
    ratio_df = pd.concat([ratio_df.iloc[:,0], pd.DataFrame(ratio_df.iloc[:,1].to_list(), columns=columns[1:])],axis=1)
    ratio_df = ratio_df.replace({"銀行別":local_dict})
    ratio_df = ratio_df.set_index("銀行別")
    ratio_df = ratio_df.replace("－", np.nan)
    ratio_df = ratio_df.applymap(lambda x:str(x)).applymap(lambda x:number_format(x))
    # 計算前20 & 總額
    month_total = []
    top_twenty = []
    for column in ratio_df:
        total = ratio_df[column].sum()
        top = ratio_df[column].nlargest(20).sum()
        month_total.append(total)
        top_twenty.append(top)
    ratio_df.loc['前20'] = top_twenty
    ratio_df.loc['總額'] = month_total

    ratio_dict[ratio_name[i]] = ratio_df

# 利用ratio_dict["指標"] 來呼叫相關dataframe, 例: ratio_dict["總存款月底餘額"]

In [22]:
print(ratio_dict["總存款月底餘額"])

               2022年1月     2022年2月     2022年3月     2022年4月     2022年5月  \
銀行別                                                                      
臺灣銀行         4215393.0   4254275.0   4316862.0   4346140.0   4415987.0   
臺灣土地銀行       2706834.0   2687970.0   2711632.0   2708190.0   2666599.0   
合作金庫銀行       3453753.0   3453818.0   3500597.0   3506185.0   3527451.0   
第一商業銀行       2964288.0   3008880.0   3054219.0   3105435.0   3104472.0   
華南商業銀行       2828722.0   2844616.0   2859143.0   2890644.0   2942953.0   
彰化商業銀行       2187738.0   2210755.0   2184282.0   2209120.0   2215740.0   
上海商業儲蓄銀行     1053254.0   1061511.0   1071661.0   1080124.0   1079638.0   
台北富邦商業銀行     2590317.0   2571026.0   2582442.0   2588302.0   2576171.0   
國泰世華商業銀行     2841037.0   2863065.0   2894865.0   2928998.0   2943072.0   
中國輸出入銀行            NaN         NaN         NaN         NaN         NaN   
高雄銀行          238937.0    237266.0    234329.0    232794.0    239183.0   
兆豐國際商業銀行     2884784.0   2911830.0   3

In [28]:
# 計算市占率及排名並把資料寫到對應dataframe的右邊
# 打開每個dataframe
for j in ratio_name:
    # 續個columns進行計算
    for i in range(1, len(columns)):
        if j == "活卡率":
            ratio_dict[j][columns[i]+"市占率"] = np.nan
            ratio_dict[j][columns[i]+"排名"] = ratio_dict[j][columns[i]].rank(ascending=False)
        elif j == "信用卡逾期帳款比率":
            ratio_dict[j][columns[i]+"市占率"] = np.nan
            ratio_dict[j][columns[i]+"排名"] = ratio_dict[j][columns[i]].rank(ascending=True)            
        else:
            ratio_dict[j][columns[i]+"市占率"] = ratio_dict[j][columns[i]]/ratio_dict[j][columns[i]].sum()  #計單月市占率並放到最右
            ratio_dict[j][columns[i]+"排名"] = ratio_dict[j][columns[i]+"市占率"].rank(ascending=False)     #計單月排名並放到最右
            ratio_dict[j][columns[i]+"市占率"] = ratio_dict[j][columns[i]+"市占率"].map(lambda x: '{:.2%}'.format(x))   #更改市占率格式

In [32]:
print(ratio_dict["總存款月底餘額"][columns[6]])

銀行別
臺灣銀行           4434159.0
臺灣土地銀行         2706520.0
合作金庫銀行         3492735.0
第一商業銀行         3098871.0
華南商業銀行         2900734.0
彰化商業銀行         2241815.0
上海商業儲蓄銀行       1105237.0
台北富邦商業銀行       2630894.0
國泰世華商業銀行       2971297.0
中國輸出入銀行              NaN
高雄銀行            227662.0
兆豐國際商業銀行       2864718.0
花旗(台灣)商業銀行      658774.0
王道商業銀行          271001.0
臺灣中小企業銀行       1664226.0
渣打國際商業銀行        579014.0
台中商業銀行          669372.0
京城商業銀行          264899.0
匯豐(台灣)商業銀行      447713.0
瑞興商業銀行           70503.0
華泰商業銀行          156367.0
臺灣新光商業銀行       1050885.0
陽信商業銀行          590186.0
板信商業銀行          257673.0
三信商業銀行          167994.0
聯邦商業銀行          676546.0
遠東國際商業銀行        606587.0
元大商業銀行         1470403.0
永豐商業銀行         1715595.0
玉山商業銀行         2734788.0
凱基商業銀行          500953.0
星展(台灣)商業銀行      388249.0
台新國際商業銀行       1774548.0
日盛國際商業銀行        234365.0
安泰商業銀行          290453.0
中國信託商業銀行       3530203.0
將來商業銀行            9438.0
連線商業銀行           36373.0
樂天國際商業銀行         17618.0
中央信託局                

In [30]:
from plot import *
# 把ratio_dict中的Dataframe儲存成csv來畫圖
store(data = ratio_dict)
plotdir(delete=False)   #畫圖, delete==True則會把df_csv資料夾刪除

成功: ['ATM裝設臺數.csv', 'OBU累計稅前損益.csv', '中小企業放款餘額.csv', '信用卡有效卡數.csv', '信用卡流通卡數.csv', '其他個人消費貸款餘.csv', '其他收益.csv', '國外分行累計稅前損益.csv', '外匯存款餘額.csv', '房貸餘額.csv', '授信總額.csv', '民營企業放款餘額.csv', '活期性存款月底餘額.csv', '流通金融卡數.csv', '消費者貸款餘額.csv', '淨公平市價變動損益.csv', '累計稅前損益.csv', '總利息收入.csv', '總存款月底餘額.csv', '總手續費收入.csv', '總放款餘額.csv', '總營業收入.csv', '金融卡交易次數.csv']
失敗: ['信用卡逾期帳款比率.csv', '活卡率.csv']


#### 電子支付（譯心）

In [ ]:
# month_list
def m_lst(yy, mm):
    year = int(yy)
    month = int(mm)
    lst = []
    [lst.append(str(year-1912) + f'{i:02d}') for i in range(month, 13)]
    [lst.append(str(year-1911) + f'{i:02d}') for i in range(1, month+1)]
    return lst

month_name_lst = m_lst(input_year, input_month)

def number_format(x):
    try:
        return int(x.replace(",", ""))
    except ValueError:
        return None

def target_months(x, y):
    year = int(x)
    month = int(y)
    ly = f'{year -1}'
    if month == 1:
        lm = '12'
        lmy = ly
    else:
        lm = f'{month-1}'
        lmy = x
    target_list = [f"{x}年{y}月", f"{lmy}年{lm}月", f"{ly}年12月", f"{ly}年{y}月"]
    return target_list

# 目錄前置：建立檔案下載資料夾
py_path = os.path.abspath('.')
folder_path = os.path.join(py_path, 'download')
zip_path = os.path.join(py_path, 'unzip')
out_path = os.path.join(py_path, 'output')
out_path_raw = os.path.join(py_path, f'output_raw_{input_year+input_month}')
if not os.path.isdir(folder_path):
    os.mkdir(folder_path)
if not os.path.isdir(out_path):
    os.mkdir(out_path)
if not os.path.isdir(zip_path):
    os.mkdir(zip_path)
if not os.path.isdir(out_path_raw):
    os.mkdir(out_path_raw)
# 選擇月份區間
# target = ['2023','1']
target = [f'{int(input_year)}', input_month]
month = [f'{int(target[0])-1911}年度', int(target[1]),
         f'{int(target[0])-1912}年度', int(target[1])]  # ['112年度',1,'111年度',1]
# month = [f'{int(input_year)-1911}年度', int(input_month), f'{int(input_year)-1912}年度', int(input_month)]  #['112年度',1,'111年度',1]

for i,j in ratio_dict.items():
    j.to_csv(out_path_raw + f"\\{i}_{target[0]}年_{target[1]}月.csv")

In [ ]:
# 更改網頁下載位置
options = Options()
options.chrome_executable_path = r'chromedriver.exe'
prefs = {"download.default_directory": folder_path}
options.add_experimental_option("prefs", prefs)

# 進入網站
driver = webdriver.Chrome(options=options)
driver.get("https://www.banking.gov.tw/ch/home.jsp?id=192&parentpath=0,4&mcustomize=multimessage_view.jsp&dataserno=201805300001&dtable=Disclosure")
driver.maximize_window()

# 下載檔案
table = driver.find_element(
    By.XPATH, '/html/body/div[1]/section/div[2]/div[3]/div[2]/div[2]/table/tbody')
for i in range(1, len(table.find_elements(By.TAG_NAME, 'tr'))+1, 2):
    year_path = driver.find_element(
        By.XPATH, f'/html/body/div[1]/section/div[2]/div[3]/div[2]/div[2]/table/tbody/tr[{i}]/th')
    year_text = year_path.text  # 年度文字
    if year_text == month[0]:
        for j in range(1, month[1]+1):
            driver.find_element(
                By.XPATH, f'/html/body/div[1]/section/div[2]/div[3]/div[2]/div[2]/table/tbody/tr[{i}]/td[{j}]').click()
    elif year_text == month[2]:
        for j in range(month[1], 13):
            driver.find_element(
                By.XPATH, f'/html/body/div[1]/section/div[2]/div[3]/div[2]/div[2]/table/tbody/tr[{i}]/td[{j}]').click()

# 關閉網頁
driver.close()

In [ ]:
# 解壓縮讀取檔案
# 讀取所有檔案名稱tolist
name_list = os.listdir(folder_path)
try:
    name_list.remove('.DS_Store')
except:
    pass

# 解壓縮
for i in name_list:
    file_name = os.path.join(folder_path, i)
    with zipfile.ZipFile(file_name, 'r') as ZIP:
        f_in_zip = ZIP.namelist()
        for file in f_in_zip:
            if file.endswith('.xlsx'):
                ZIP.extract(file, path=zip_path)
# 刪除壓縮檔
    os.remove(file_name)
os.rmdir(folder_path)

In [ ]:
# 讀取excel檔
# 檔案重命名：11101  *111年1月*
excel_list = os.listdir(zip_path)
for i in excel_list:
    os.rename(f'./unzip/{i}', f'./unzip/{i[:5]}.xlsx')
excel_list = os.listdir(zip_path)

In [ ]:
# 輸入為dataframe
df_lst = []
電子支付使用者人數 = pd.DataFrame()
電子支付當月代理收付實質款項交易金額 = pd.DataFrame()
電子支付支付款項餘額 = pd.DataFrame()
df_lst = [電子支付使用者人數, 電子支付當月代理收付實質款項交易金額, 電子支付支付款項餘額]
df_lst_text = ['電子支付使用者人數', '電子支付當月代理收付實質款項交易金額', '電子支付支付款項餘額']

for i in month_name_lst:
    excel = pd.read_excel(f"./unzip/{i}.xlsx")
    excel = excel.drop(excel.columns.tolist()[3:-1], axis=1)
    excel.columns = ['bank', '電子支付使用者人數', '電子支付當月代理收付實質款項交易金額', '電子支付支付款項餘額']

    twbank = excel.index[excel['bank'] == '臺灣銀行'].tolist()[0]
    total = excel.index[excel['bank'] == '總計'].tolist()[0]
    excel = excel.iloc[twbank:total, :]

    temp_user = pd.DataFrame(excel.iloc[:, 1].tolist(), columns=[
                             '2022年1月'], index=excel.iloc[:, 0].tolist())
    temp_money = pd.DataFrame(excel.iloc[:, 2].tolist(), columns=[
                              '2022年1月'], index=excel.iloc[:, 0].tolist())
    temp_balance = pd.DataFrame(excel.iloc[:, 3].tolist(), columns=[
                                '2022年1月'], index=excel.iloc[:, 0].tolist())

    for j in range(3):
        change_month_format = f'{int(i[:3])+1911}年{int(i[3:5])}月'
        temp_df = pd.DataFrame(
            excel.iloc[:, j+1].tolist(), columns=[change_month_format], index=excel.iloc[:, 0].tolist())
        df_lst[j] = pd.merge(df_lst[j], temp_df, how='outer',
                             left_index=True, right_index=True)

# 刪除檔案
    os.remove(f"./unzip/{i}.xlsx")
os.rmdir(zip_path)


In [ ]:
# 編寫為csv
# 計算前20與排名
for i in df_lst:
    month_total = []
    top_twenty = []
    for column in i:
        total = i[column].sum()
        top = i.nlargest(20, column)[column].sum()
        month_total.append(total)
        top_twenty.append(top)

    i.loc['前20'] = top_twenty
    i.loc['總額'] = month_total

# market share & rank
for i in df_lst:
    # df_lst[i] = df_lst[i].reset_index().rename(columns={"index":"銀行別"})
    columns = i.columns.tolist()
    for j in range(len(columns)):
        i[columns[j]+"市占率"] = i[columns[j]]/i[columns[j]].sum()
        i[columns[j]+"排名"] = i[columns[j]].rank(ascending=False)
        i[columns[j]+"市占率"] = i[columns[j] +
                                "市占率"].map(lambda x: '{:.2%}'.format(x))

# 將電子支付部分存入所有指標的dictionary
for i in range(len(df_lst)):
    ratio_dict[df_lst_text[i]] = df_lst[i]

將各指標寫入csv檔

In [ ]:
# growth rate
temp = 0
for key, value in ratio_dict.items():
    w = key
    table = value
    # 建立欲輸出的df
    df = pd.DataFrame()

    target_month = target_months(target[0], target[1])
    df = table[target_month]

    # 若有重複月份
    for i in range(0, 4):
        for j in range(i+1, 4):
            if target_month[i] == target_month[j]:
                target_month[j] = f'{target_month[j]}_1'
    df.columns = target_month

    # 計算市占率&排名
    for column in range(0, df.shape[1] * 3, 3):
        market_share = []
        for row in range(df.shape[0]):
            share = df.iloc[row, column] / df.iloc[df.shape[0] - 1, column]
            market_share.append(share)

        ranking = df[target_month[column // 3]].iloc[:-2].rank(ascending=False)

        df.insert(
            column + 1, column=f'{target_month[column // 3]}市占率', value=market_share)
        df.iloc[:, column + 1] = df.iloc[:, column +
                                         1].map(lambda x: '{:.2%}'.format(x))
        df.insert(
            column + 2, column=f'{target_month[column // 3]}排名', value=ranking)

    # 與前__比較
    new_target = ['上月增量', '去年年底增量', '去年同期增量']
    [target_month.append(i) for i in new_target]

    df['上月增量'] = df[target_month[0]] - df[target_month[1]]
    df['去年年底增量'] = df[target_month[0]] - df[target_month[2]]
    df['去年同期增量'] = df[target_month[0]] - df[target_month[3]]

    for column in range(12, 21, 3):
        ranking = df[target_month[column // 3]].iloc[:-2].rank(ascending=False)
        rate = df[target_month[column // 3]]/df[target_month[0]]
        df.insert(
            column + 1, column=f'{target_month[column // 3]}成長率', value=rate)
        df.iloc[:, column + 1] = df.iloc[:, column +
                                         1].map(lambda x: '{:.2%}'.format(x))
        df.insert(
            column + 2, column=f'{target_month[column // 3]}排名', value=ranking)

    # 寫入csv(1): dataframe
    out = df.sort_values(by=target_month[0], ascending=False)
    out = out.head(22)
    out = out.sort_values(by=[f'{target_month[0]}排名', f'{target_month[0]}市占率'])
    if (key == "活卡率") or (key == "信用卡逾期帳款比率"):
        out.iloc[:, [0, 3, 6, 9]] = out.iloc[:, [0, 3, 6, 9]
                                             ].applymap(lambda x: '{:.2%}'.format(x))
        out.iloc[:, [1, 4, 7, 10]] = "-"
    # print(df)
    col = out.columns.values.tolist()
    col.insert(0, '')
    bank_name = out.index.values.tolist()

    file_path = os.path.join(out_path, f'{w}_{target[0]}年{target[1]}月.csv')

    with open(file_path, 'w', encoding='utf-8-sig', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(col)
        for i in range(out.shape[0]):
            inform = out.iloc[i].tolist()
            inform.insert(0, bank_name[i])
            writer.writerow(inform)
        f.close()

    # 寫入csv(2): 玉山
    main_file = os.path.join(out_path, "玉山銀行彙整.csv")
    # col = out.columns.values.tolist()
    # col.insert(0, '')
    esun = df.loc['玉山商業銀行'].tolist()
    esun.insert(0, w)

    if os.path.isfile(main_file):
        with open(main_file, 'a', encoding='utf-8-sig', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(esun)
            f.close()

    else:
        with open(main_file, 'a', encoding='utf-8-sig', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(col)
            writer.writerow(esun)
            f.close()

    temp += 1
